In [29]:
from scipy.special import expit
import numpy as np

In [30]:
class ANN(object):
    def __init__(self, innum, outnum, lr, *hide_tuple):
        self.innum = innum                   # 输入节点的个数
        self.outnum = outnum                 # 输出节点的个数
        self.lr = lr                         # 学习率
        self.layernum = len(hide_tuple) + 1  # 神经网络的层数
        self.Weight = {}                     # 权重矩阵
        self.Bias = {}                       # 偏置项

        #　对权重矩阵和偏置项进行初始化
        self.Weight[1] = np.random.normal(0.0, pow(self.innum, -0.5), (hide_tuple[0], self.innum))
        for i in range(1, self.layernum):
            self.Bias[i] = np.random.randn(hide_tuple[i-1]).reshape(hide_tuple[i-1], 1)
            if i>=2:
                self.Weight[i] = np.random.normal(0.0, pow(hide_tuple[i-1], -0.5), (hide_tuple[i-1], hide_tuple[i-2]))
        self.Weight[self.layernum] = np.random.normal(0.0, pow(hide_tuple[-1], -0.5), (self.outnum, hide_tuple[-1]))
        self.Bias[self.layernum] = np.random.randn(self.outnum).reshape(self.outnum, 1)

        self.ActiveFunction = lambda x: expit(x)  # 激活函数为logistic函数

    def BPFit(self, input_list, target_list):
        Input0 = np.array(input_list, ndmin=2).T
        TargetValue = np.array(target_list, ndmin=2).T

        Input = {}      # 输入值
        Output = {}     # 输出值
        Output[0] = Input0
        for i in range(1, self.layernum+1):
            Input[i] = np.dot(self.Weight[i], Output[i-1])
            Output[i] = self.ActiveFunction(Input[i] + self.Bias[i])

        Error={}        # 误差项
        Error[self.layernum] = Output[self.layernum] * (1 - Output[self.layernum]) * (-(TargetValue - Output[self.layernum]))
        self.Weight[self.layernum] -= self.lr * Error[self.layernum] * Output[self.layernum-1].T
        self.Bias[self.layernum] -= self.lr * Error[self.layernum]
        for i in range(self.layernum-1, 0, -1): # 从倒数第二层的误差项开始
            Error[i] = Output[i] * (1 - Output[i]) * np.dot(self.Weight[i+1].T, Error[i+1])
            self.Weight[i] -= self.lr * Error[i] * Output[i-1].T
            self.Bias[i] -= self.lr * Error[i]

    def predict(self, input_list):
        Input0 = np.array(input_list, ndmin=2).T
        Input = {}
        Output = {}
        Output[0] = Input0
        for i in range(1, self.layernum + 1):
            Input[i] = np.dot(self.Weight[i], Output[i - 1])
            Output[i] = self.ActiveFunction(Input[i] + self.Bias[i])

        return Output[self.layernum]

In [31]:
if __name__ == '__main__':

    """
    测试样例
    """

    inode = 4           # 输入节点个数
    hnode1 = 4          # 第1层隐节点个数
    hnode2 = 5          # 第2层隐节点个数
    hnode3 = 10         # 第3层隐节点个数
    hnode4 = 5          # 第4层隐节点个数
    onodenum = 1        # 输出节点个数
    learningrate = 0.3  # 学习率
    ann = ANN(inode, onodenum, learningrate, hnode1, hnode2, hnode3, hnode4)

    TrainValue = [1, 3, 2, 4]
    TargetValue = [0]
    ann.BPFit(TrainValue, TargetValue)

    TestValue = [2, 4, 2, 4]
    predict = ann.predict(TestValue)
    print(predict)      # 输出预测结果

[[0.63055207]]
